In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Pandas (again)

URL https://github.com/FIIT-IAU


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
filename = 'data/sales-funnel.csv'
df = pd.read_csv(filename)
df.head()

## Deskriptívna štatistika

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

**Koľko riadkov má cenu vyššiu ako $8,000?**

In [ ]:
df.Price > 8000

In [ ]:
len(df[df.Price > 8000])

Keďže nás nezaujímajú samotné riadky, ale len ich počet, tak rovnaký výsledok sa dá dosiahnuť aj efektívnejšie. Spočítaním koľko krát je podmienka splnená. Vtedy netreba získavať dáta a pri veľkých datasetoch to môže učetriť veľmi veľa času a zdrojov. 

In [ ]:
sum(df.Price > 8000)

Vykonanie podmienky vráti binárny vektor. True hodnoty sa považujú pri operácii sum za 1 a False za 0. Sumovaním teda získame počet riadkov, kde bola podmienka splnená.

**Koľko riadkov má status "pending" a cenu vyšiu ako $8,000?**

In [ ]:
sum((df.Status == 'pending') & (df.Price > 8000))

**Koľko riadkov má status "pending" alebo cenu vyšiu ako $8,000?** 

In [ ]:
sum((df.Status == 'pending') | (df.Price > 8000))

**Vytvorte stĺpec s názvom "amount", ktorý bude tvorený násobkom stĺpcpv "price" a "quantity".**
Aký je súčet hodnôt v tomto stĺpci pre riadky so statusom "won"?

In [ ]:
df.Price * 2

In [ ]:
df['amount'] = df.Price * df.Quantity
df.head()

In [ ]:
df[df.Status == 'won'].amount.sum()

**Aký je celkový súčet stĺpcu "amount" pre záznamy produktov z kategórie "CPU"?**

In [ ]:
df[df.Product == 'CPU'].amount.sum()

# Zmena tvaru tabuliek

## Pivot table

**Aký je rozdiel medzi funkciami "pivot" a "pivot_table"?**

pivot_table umožňuje definovať agregačnú funkciu, ktorá spája hodnoty pri duplicitných kombináciách v indexe a poemnovaniach stĺpcov.

**V dátovej sade z predchádzajúcich úloh vytvorte tabuľku, kde v riadkoch budú záznamy pre rôzne produkty, v stĺpcoch záznamy pre rôzne statusy a hodnoty budú agregované hodnoty zo stĺpcu "amount".** Na agregáciu použite funkciu sum z balíčka numpy.

In [ ]:
df

In [ ]:
pd.pivot_table(df, values='amount', index='Product', columns='Status', aggfunc=np.sum)

## Groupby

In [ ]:
df.groupby(['Product', 'Status']).amount.sum().unstack().unstack().unstack()

## Cross tab
**Vytvorte kontingenčnú tabuľku (crosstab), ktorá ukáže koľko záznamov je v jednotlivých stavoch (status) pre jednotlivé produkty**

In [ ]:
pd.crosstab(df.Product, df.Status)

**Vytvorte tabuľku z prvej úlohy, ale nahraďte produkty za Manažérov alebo Rep alebo dvojice Manažér a Rep. Vzniknú vám teda 3 rôzne tabuľky.**

In [ ]:
pd.crosstab(df.Manager, df.Status)

In [ ]:
pd.crosstab(df.Rep, df.Status)

In [ ]:
pd.crosstab([df.Manager, df.Rep], df.Status)

## Iný dataset
### 1. Načítajte dataset - pozor: kódovanie "latin1"

In [ ]:
df2 = pd.read_csv('data/crunchbase_monthly_export.csv', encoding='latin1')
df2.head()

### 2. Deskriptívna štatistika

Ide o dataset informácií o firmách/projektoch, ktoré žiadali o investíciu a o tom, či boli v žiadosti úspešní

In [ ]:
df2.info()

Dataset má veľa stĺpcov obsahujúcich reťazce aj keď by sme tam logicky očakávali číslo. Obsahuje tiež celkom vela chýbajúcich hodnôt.  

In [ ]:
df2.describe(include='all')

### 3. Aký je maximálny získaný funding? 
Pomôcka: budete musieť transformovať a upraviť hodnoty. Poriadne sa pozrite na dátový typ v stĺpci a na názov stĺpca.

In [ ]:
df2.columns

niektoré stĺpce majú v názvoch medzery na začiatku a na konci

In [ ]:
df2[' funding_total_usd '].head(10)

sledovaný stĺpec obsahuje reťazce a viaceré nechcené symboly, ktoré bude treba odstrániť

In [ ]:
# df2[' funding_total_usd '] = pd.to_numeric(df2[' funding_total_usd '].str.replace(r'\D', ''))
df2[' funding_total_usd '] = pd.to_numeric(df2[' funding_total_usd '].str.replace(r'\D', '', regex=True))

**Pozor**, v predchádzajúcej bunke robím nebezpečnú operáciu. Mením hodnoty toho istého stĺpca, ktorý práve spracovávam. 

**Po správnosti by som mal vytvoriť nový stĺpec s iným názvom.** Ak totiž táto transformácia spraví niečo neočakávané, tak si nezachovám pôvodnú kópiu dát a nebudem ju vedieť spustiť odznovu.  

### 4. Premenujte stĺpce tak aby boli zrozumiteľné a správne formátované.

In [ ]:
df2 = df2.rename(columns=str.strip)
df2.columns

### 5. Vytvorte tabuľku hodnôt zo stĺpcu "Market" a súčtu hodnôt total funding pre záznamy s danou hodnotou v stĺpci "Market". Usporiadajte tieto hodnoty od najväčšej po menjmenšiu a zobrazte prvých 10 z nich.

In [ ]:
df2.groupby('market')\
    .funding_total_usd.sum()\
    .reset_index()\
    .sort_values(by='funding_total_usd', ascending=False)\
    .head(10)

### 6. Koľko spoločností obsahuje kategóriu "Game"?

In [ ]:
sum(df2.category_list.apply(lambda x: 'Game' in str(x).split('|')))

### 7. Aký je priemerný total funding pre spoločnosti z New Yorku? zo San Francisca? Porovnajte tieto hodnoty

In [ ]:
df2[df2.city == 'New York'].funding_total_usd.mean()

In [ ]:
df2[df2.city == 'San Francisco'].funding_total_usd.mean()

## Uloha na cvicenie

dataset: https://www.kaggle.com/fivethirtyeight/fivethirtyeight-bad-drivers-dataset

In [ ]:
import seaborn as sns

In [ ]:
cars = sns.load_dataset('car_crashes')

In [ ]:
cars.head()

#### 1. zakladne vlastnosti, deskriptivne statistiky, etc

#### 2. zakladne grafy distribucii, skuste najst korelacie (predovsetkym s celkovym poctom vodicov s fatalnymi nehodami)

#### 3. najdite stat s najvacsim / najmensim poctom nehod; urcite priemerny pocet pre jednotlicve krajiny

#### 4. rozdelte podla poctu nehod rovnomerne na 4 skupiny, pre kazdu skupinu uvedte priemerne hodnoty vsetkych ostatnych stlpcov